In [ ]:
from zipfile import ZipFile
from PIL import Image, ImageOps
import numpy as np
import json

with open("morse.json", "r" ) as m:
    morsecode = json.load(m)

In [ ]:
def decode_password( image ):
    img         = ImageOps.grayscale( ImageOps.autocontrast( Image.open( image, mode="r") ) )
    img.save( image+"gray.png"  )
    sizeX,sizeY = img.size
    ndImg       = np.asarray(img)
    ndImg       -= ndImg[0][0]
    morseLines  = ["",]*int((sizeY-1)/2)
    
    print( ndImg )

    yCount = 0
    for imgLine in ndImg[1::2]:
        morseLine = ""
        count = 0
        for i in imgLine[1:-1]:
            if i != 0:
                count += 1
            elif i == 0 and count > 0:
                morseLine += "." if count == 1 else "-"
                count = 0
        morseLines[yCount] = morseLine
        yCount += 1
    
    pwd = ""
    for morseLine in morseLines:
        if morseLine in morsecode:
            pwd += morsecode[morseLine]
    return bytes(pwd.lower(), 'utf-8')

In [ ]:
def parseArchive( fname:str="", passwd="", dest="" ):
    newZip = False
    cFiles = []
    with ZipFile( fname, "r" ) as zf:
        cFiles = zf.namelist()
        for cFile in cFiles:
            if cFile.replace("flag/","").startswith("flag_"):
                newZip = True
            zf.extractall( path=dest, pwd=passwd)
    return newZip, cFiles

cont        = True
archCount   = 2000
fname = "M0rsarchive.zip"
passwd = b"hackthebox"

fname = "unzipped_1121/flag/flag_878.zip"
passwd = b'ggimk7poj29vtckcud5qyy365v'

while cont:
    cont, files = parseArchive( fname, passwd, "unzipped_{}".format(archCount) )
    if not cont:
        print("MATCH")
        print(files)
        print(  "unzipped_{}".format(archCount)  )
        break
    for fn in files:
        if ".zip" in fn:
            fname = "unzipped_{}/{}".format(archCount, fn)
        elif ".png" in fn:
            passwd = decode_password( "unzipped_{}/{}".format(archCount, fn) )
    print( fname, passwd )
    archCount += 1
    print( "-"*20 )